In [1]:
import pandas as pd
import os, glob, re

from owlready2 import *
import owlready2
print(owlready2.VERSION)


import importlib.util
import sys
spec = importlib.util.spec_from_file_location("rdfutils", "../../../utils/rdfutils.py")
u = importlib.util.module_from_spec(spec)
sys.modules["rdfutils"] = u
spec.loader.exec_module(u)

spec = importlib.util.spec_from_file_location("llm", "../../../utils/llm.py")
h = importlib.util.module_from_spec(spec)
sys.modules["llm"] = h
spec.loader.exec_module(h)

from datetime import datetime

def NOW():
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    return "Current Time = "+ str(current_time)

%load_ext autoreload
%autoreload 2


0.40


* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
import requests, json

In [3]:
onto = get_ontology("./WIP_0.owl").load()
comments = u.checkComments(onto)

ID: 0 	 Description: Creation of a knowledge graph based on a litterature review, augmented by use of LLMs.
ID: 1 	 Version: 0.4
ID: 2 	 Library: owlready2==0.45
ID: 3 	 Changes from: 0.3
ID: 4 	 Next: Check formulas for selecting most repeated items in groups
ID: 5 	 Changes: Adding new risks, groups of items, and new relationships
ID: 6 	 Creation: 22/01/2024
ID: 7 	 TODOs: Adding synonyms and solving classification with synonyms
ID: 8 	 VersionComment: Grouping of items added in 0.4
ID: 9 	 Project: PROBONO
ID: 10 	 Next: Linking benefits to mitigations groups
ID: 11 	 License: CC BY-NC-SA
ID: 12 	 Task: T3.5
ID: 13 	 Repository: https://github.com/mm80843/T3.5/
ID: 14 	 Author: Luc Jonveaux
ID: 15 	 Language: English


# Asking the BOK API.

This is the one in the current report, in the bok/data/main_api.py FastAPI app.

In [4]:
with onto:
    class has_Description(onto.PBNThing >> str):
        label = ["Short description"]
        pass
    class has_Note(onto.PBNThing >> str):
        label = ["Author note"]
        pass

In [5]:
onto.BenefReturn.label = ["Benefit returns"]
onto.Benef.label = ["Benefit"]
onto.Article.label = ["Research article"]
onto.Risk.label = ["Risk"]
onto.ISO_Scale.label = ["Scale in the ISO37100 (GBN) reference system"]
onto.ISO_Scale.has_Note = ["Used according to the ISO 37100 exercice to assess the scale an item in a GBN."]
onto.RiskHealth.label = ["Type of health impact"]
onto.RiskType.label = ["Type of the risk"]
onto.Stakeholder.label = ["Stakeholder"]
onto.Stakeholder_Type.label = ["Type of stakeholder"]
onto.Technology.label = ["Technology"]
onto.ISO_Impact.label = ["Impact in the ISO37100 (GBN) reference system"]
onto.ISO_Purpose.label = ["Purpose in the ISO37100 (GBN) reference system"]
onto.StakeholderGroup.label = ["Group of stakeholders"]
onto.StakeholderSubgroup.label = ["Subroup of stakeholders"]
onto.TechGroup.label = ["Group of technologies"]
onto.TechSubgroup.label = ["Subroup of technologies"]
onto.Mitigation.label = ["Risk mitigation"]
onto.BP_Enabler.label = ["Blueprint: enabler"]
onto.BP_Transmission.label = ["Blueprint: transmission mode"]
onto.Blueprint.label = ["Blueprint"]
onto.BP_Scale.label = ["Blueprint: scale"]
onto.BP_Phase.label = ["Blueprint: phase"]
onto.BP_Permanent.label = ["Blueprint: is it permanent?"]
onto.BP_Type.label = ["Blueprint: Type"]
onto.BP_Intervention.label = ["Blueprint: intervention family"]
onto.PBNCategory.label = ["Category of objects"]
onto.RiskGroup.label = ["Group of risks"]
onto.RiskSubgroup.label = ["Subgroup of risks"]
onto.CAO_Group.label = ["Localisation: City Anatomoy level 1"]
onto.CAO_Group.has_Note = ["For use with the City Anatomy ontology.\n See page 26 of https://s3.amazonaws.com/nist-sgcps/smartcityframework/files/GCTCTechJamKickoff/JimAloisi_NIST_IES_city_presentation_jarecov_3_2016-2.pdf ."]
onto.CAO_Subgroup.label = ["Localisation: City Anatomoy level 2"]
onto.CAO_Subgroup.has_Note = ["For use with the City Anatomy ontology.\n See page 26 of https://s3.amazonaws.com/nist-sgcps/smartcityframework/files/GCTCTechJamKickoff/JimAloisi_NIST_IES_city_presentation_jarecov_3_2016-2.pdf ."]
onto.BP_Theme.label = ["Blueprint: Theme"]
onto.BP_Category.label = ["Blueprint: Category"]
onto.aBlueprint.label = ["Blueprint, by genAI"]
onto.aBlueprint.has_Note = ["For inspiration only. Generated by AI, not reviewed by humans."]

In [6]:
A = list(onto.classes())[2]
len(A.instances())

1409

# Saving

In [7]:
onto.save("WIP_1.owl")

In [8]:
print("Done")

Done
